# Proyek Deteksi Warna Pada Gambar
Tujuan dari proyek ini adalah untuk membuat sebuah model yang dapat mendeteksi warna pada inputan gambar dari user, untuk warna yang ada berjumlah 15 karena dataset ini saya dapatkan dari platform [Kaggle](https://www.kaggle.com/datasets/adikurniawan/color-dataset-for-color-recognition) yang dimana di dalam dataset tersebut terdapat 10 folder dengan isi warna gambar masing-masing folder berjumlah 25, karena merasa kurang saya pun membuat sendiri mengambil gambar warna yang ada di google berjumlah 5 folder dengan masing-masing folder berisi 25 gambar.

### 1. Setup

In [ ]:
import tensorflow as tf

In [ ]:
!pip install split_folders tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders

splitfolders.ratio('/content/drive/MyDrive/Dataset/color_image', output='data', seed=3, ratio=(.8, .2))

Copying files: 375 files [02:51,  2.18 files/s]


1.1 Load data

In [ ]:
import os

data = '/content/data'
train_dir = os.path.join(data, 'train')
validation_dir = os.path.join(data, 'val')

### 2. Preprocess the data

2.1 Scale data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator                        

training_datagen = ImageDataGenerator(                                           
    rescale = 1./255,
    zoom_range = 0.2,
    rotation_range = 20,
    shear_range = 0.2,
    fill_mode = 'nearest'
)

2.2 Split data

In [ ]:
train_generator = training_datagen.flow_from_directory(                           
    train_dir, 
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical'
)

validation_generator = training_datagen.flow_from_directory( 
    validation_dir,
    target_size=(150, 150),
    batch_size=16, 
    class_mode='categorical'
)

Found 300 images belonging to 15 classes.
Found 75 images belonging to 15 classes.


### 3. Bangun arsitektur model menggunakan CNN






In [ ]:
import tensorflow_hub as hub

model = tf.keras.models.Sequential([
    # hub.KerasLayer(
    #     'https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2',
    #     input_shape=(150, 150, 3),
    #     trainable=False
    # ),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(512, activation='relu'),
    # tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax') 
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

### 4. Latih Model

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') > 0.90 and logs.get('val_accuracy') > 0.90):
      print("\nPELATIHAN BERHENTI, AKURASI MODEL SUDAH LEBIH DARI 90%!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
history = model.fit(train_generator,
    validation_data=validation_generator,
    epochs=500,
    verbose=2,
    callbacks=[callbacks]
  )

Epoch 1/500


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


38/38 - 23s - loss: 2.1053 - accuracy: 0.3133 - val_loss: 1.7877 - val_accuracy: 0.4800 - 23s/epoch - 605ms/step
Epoch 2/500
38/38 - 5s - loss: 1.3073 - accuracy: 0.5500 - val_loss: 0.9482 - val_accuracy: 0.6667 - 5s/epoch - 139ms/step
Epoch 3/500
38/38 - 4s - loss: 1.2947 - accuracy: 0.5367 - val_loss: 1.0861 - val_accuracy: 0.5467 - 4s/epoch - 97ms/step
Epoch 4/500
38/38 - 4s - loss: 0.9482 - accuracy: 0.6067 - val_loss: 0.8697 - val_accuracy: 0.6133 - 4s/epoch - 97ms/step
Epoch 5/500
38/38 - 5s - loss: 0.9004 - accuracy: 0.6167 - val_loss: 0.9015 - val_accuracy: 0.6800 - 5s/epoch - 137ms/step
Epoch 6/500
38/38 - 4s - loss: 0.8467 - accuracy: 0.6633 - val_loss: 0.7825 - val_accuracy: 0.7067 - 4s/epoch - 96ms/step
Epoch 7/500
38/38 - 5s - loss: 0.6762 - accuracy: 0.7300 - val_loss: 0.7484 - val_accuracy: 0.7067 - 5s/epoch - 133ms/step
Epoch 8/500
38/38 - 4s - loss: 0.6906 - accuracy: 0.7267 - val_loss: 0.7764 - val_accuracy: 0.6800 - 4s/epoch - 98ms/step
Epoch 9/500
38/38 - 4s - loss:

4.1 Evaluasi

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

5/5 - 1s - loss: 0.3403 - accuracy: 0.9067 - 683ms/epoch - 137ms/step
Test loss: 0.34034186601638794
Test accuracy: 0.9066666960716248


# 5. Save model

In [ ]:
# TF JS
model.save('color_detection_model.h5')

# TF Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with tf.io.gfile.GFile('color_detection_model.tflite', 'wb') as f:
  f.write(tflite_model)

print("Model telah disimpan.")

Model telah disimpan.


# Percobaan color detection

In [ ]:
!pip install gradio
import gradio as gr
import numpy as np
from numpy import asarray
from datetime import datetime
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Dataset/color_detection_model.h5")
# model = tf.keras.models.load_model("/content/drive/MyDrive/Dataset/color_detection_model.hdf5")

def image_predict(img):
    """
    Displays dominant colors beyond a given threshold.
    img : image input, for ex 'blue-car.jpg'
    isize: input image size, for ex. 227
    thr: chosen threshold value
    """
    thr=0
    global model
    if model is None:
        model = tf.keras.models.load_model("/content/drive/MyDrive/Dataset/color_detection_model.h5")
        # model = tf.keras.models.load_model("/content/drive/MyDrive/Dataset/color_detection_model.hdf5")
        
    data = np.asarray(img)
        
    ndata = np.expand_dims(data, axis=0)
    y_prob = model.predict(ndata/255)
    #y_prob.argmax(axis=-1)
    
    now = datetime.now()
    print("--------")
    print("data and time: ", now)
            
    colorlabels = ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']
    coltags = [sorted(colorlabels)[i] for i in np.where(np.ravel(y_prob)>thr)[0]]
    colprob = [np.ravel(y_prob)[i] for i in list(np.where(np.ravel(y_prob)>thr)[0])]
    
    if len(coltags) > 0:
        response = []
        for i,j in zip(coltags, colprob):
            #print(i,j)
            resp = {}
            resp[i] = float(j)
            response.append(resp)
        d = dict(map(dict.popitem, response))
        print('colors: ', d)
               
        return dict(d)

    else:
        return str('No label was found')

In [ ]:
iface = gr.Interface(
    title = "Color Image Detecting",
    description = "App classifying objects on different colors",
    article = "<p style='text-align: center'><a href='https://www.rrighart.com' target='_blank'>Webpage</a></p>",
    fn=image_predict,
    inputs=gr.Image(shape=(150,150)), 
    outputs=gr.Label(),
    enable_queue=True,
    interpretation="default",
    debug=True
    )
iface.launch()

<ipython-input-21-fb1a86752e09>:1: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  iface = gr.Interface(
<ipython-input-21-fb1a86752e09>:1: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'debug': True}
  iface = gr.Interface(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/color_detection_model.hdf5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflmodel = converter.convert()
file = open( 'color_detection_model2.tflite' , 'wb' ) 
file.write( tflmodel )

233390768